<a href="https://colab.research.google.com/github/Palemravichandra/CRUD-Operations-with-CSV-file/blob/main/vengage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import csv
import json
import re
import string
import pandas as pd

class PhoneBook:
    def __init__(self):
        self.records = []
        #print(type(self.records))

    def load_records_from_file(self, file_path):
        extension = file_path.split('.')[-1]
        if extension == 'csv':
            self._load_csv(file_path)
        elif extension == 'json':
            self._load_json(file_path)
        else:
            raise ValueError("Unsupported file format")

    def _load_csv(self, file_path):
        df=pd.read_csv(file_path)
        #print(df)
        self.records=df.to_dict(orient='records')
        #print(self.records)
    def strip_chars_except_at(input_string):
        pattern = r'[) ,;](?<!@)'
        # Use re.sub() to replace matches with an empty string
        result_string = re.sub(pattern, '', input_string)
        return result_string

    def execute_query(self, query):
        # Implementing a simple SQL-like parser
        tokens = query.split()
        command = tokens[0].upper()

        if command == 'SELECT':
            return self._select_records(tokens)
        elif command == 'INSERT':
            return self._insert_record(tokens)
        elif command == 'DELETE':
            return self._delete_record(tokens)
        else:
            raise ValueError("Unsupported command")

    def _select_records(self, tokens):
      if tokens[1] == '*'and len(tokens)==4:
          df=pd.DataFrame(self.records[:10]) # Display the first 10 records
          return df
      elif tokens[1] == 'FROM' and tokens[2] == 'phone_records' and tokens[3] == 'WHERE':
        field,val = tokens[4].split('=')
        value=strip_chars_except_at(val).replace("'",'')
        df=pd.DataFrame([record for record in self.records if record[field]== value])
        return df

      elif tokens[1] == '*' and tokens[2] == 'FROM' and tokens[3] == 'phone_records' and tokens[4]=='WHERE':
        field,val = tokens[5].split('=')
        value=strip_chars_except_at(val).replace("'",'')
        #return field,value
        df=pd.DataFrame(self.records[:])
        return df[df[field]==value]
      else:
          raise ValueError("Invalid SELECT query")

    def _insert_record(self, tokens):
      if len(tokens)<8  or tokens[1] != "INTO" :
        #print(tokens)
        return "Invalid INSERT statement"
      name = tokens[2]
      table_name=name[0:13]
      if table_name != "phone_records":
          return "Table not found: {}".format(table_name)

      field_names=str(tokens[2:6])
      value = strip_chars_except_at(field_names).replace("'", ' ')
      #print(value)
      field_names=value[16:45].split()
      #print(k)
      values_start = tokens[6:]
      k=values_start[0]
      k=k[8:]
      values_start[0]=k
      #print(type(values_start))
      values=[]
      for i in values_start:
        B3=strip_chars_except_at(i).replace("'",' ').strip()
        values.append(B3)
      new_record = dict(zip(field_names, values))
      self.records.append(new_record)
      return f"{new_record}'Record inserted successfully"


    def _delete_record(self, tokens):
        if tokens[1] == 'FROM' and tokens[2] == 'phone_records' and tokens[3] == 'WHERE':
            field, val = tokens[4].split('=')
            value=strip_chars_except_at(val).replace("'",'')
            deleted_records = [record for record in self.records if record[field].lower() == value.lower()]
            self.records = [record for record in self.records if record[field].lower() != value.lower()]
            return f"Records deleted: {deleted_records}"
        else:
            raise ValueError("Invalid DELETE query")



# Example usage:
phone_book = PhoneBook()
phone_book.load_records_from_file("phonebook.csv")



In [172]:
phone_book.execute_query("SELECT * FROM phone_records;")

,Name,email,Phone_1,Phone_2
0,Ravi,ravi@gmail,2288999992,2233445566
1,Raja,raja@gmail,1122334455,2211332233
2,Rani,rani@gmail,213221232,2121232323
3,Kani,Kani@gmail,3234342343,2312321234
4,Roja,Roja@gmail,3455432342,5673564711
5,Jon,Jon@gmail,3245671233,27364858121
6,Raghu,Raghu@gmail,2345321234,9808080090
7,John,John@gmail,2277881199,2637291899
8,Karan,Karan@gmail,4256178926,3729128291
9,Baby,Baby@gmail,2233441122,2343212341


In [173]:
phone_book.execute_query("SELECT * FROM phone_records WHERE Name='Ravi;")

,Name,email,Phone_1,Phone_2
0,Ravi,ravi@gmail,2288999992,2233445566
11,Ravi,ravi@gmail,3435563223,2234354599
12,Ravi,ravi@gmail,2312321222,3348392200


In [174]:
result3 = phone_book.execute_query("INSERT INTO phone_records(Name, email, Phone_1, Phone_2) VALUES('King', 'King@abc.com', '13465789', '123323334');")
print(result3)

{'Name': 'King', 'email': 'King@abc.com', 'Phone_1': '13465789', 'Phone_2': '123323334'}'Record inserted successfully


In [175]:
phone_book.execute_query("SELECT * FROM phone_records;")

,Name,email,Phone_1,Phone_2
0,Ravi,ravi@gmail,2288999992,2233445566
1,Raja,raja@gmail,1122334455,2211332233
2,Rani,rani@gmail,213221232,2121232323
3,Kani,Kani@gmail,3234342343,2312321234
4,Roja,Roja@gmail,3455432342,5673564711
5,Jon,Jon@gmail,3245671233,27364858121
6,Raghu,Raghu@gmail,2345321234,9808080090
7,John,John@gmail,2277881199,2637291899
8,Karan,Karan@gmail,4256178926,3729128291
9,Baby,Baby@gmail,2233441122,2343212341


In [176]:
result4 = phone_book.execute_query("DELETE FROM phone_records WHERE Name='Ravi';")
print(result4)


Records deleted: [{'Name': 'Ravi', 'email': 'ravi@gmail', 'Phone_1': 2288999992, 'Phone_2': 2233445566}, {'Name': 'Ravi', 'email': 'ravi@gmail', 'Phone_1': 3435563223, 'Phone_2': 2234354599}, {'Name': 'Ravi', 'email': 'ravi@gmail', 'Phone_1': 2312321222, 'Phone_2': 3348392200}]


In [177]:
phone_book.execute_query("SELECT * FROM phone_records;")

,Name,email,Phone_1,Phone_2
0,Raja,raja@gmail,1122334455,2211332233
1,Rani,rani@gmail,213221232,2121232323
2,Kani,Kani@gmail,3234342343,2312321234
3,Roja,Roja@gmail,3455432342,5673564711
4,Jon,Jon@gmail,3245671233,27364858121
5,Raghu,Raghu@gmail,2345321234,9808080090
6,John,John@gmail,2277881199,2637291899
7,Karan,Karan@gmail,4256178926,3729128291
8,Baby,Baby@gmail,2233441122,2343212341
9,Gnana,Gnana@gmail,2233112343,2628618161
